# Welcome to the 
```
   _  _____  ___  ___  _____________  ____  _  __
  / |/ / _ \/ _ \/ _ \/  _/_  __/ _ \/ __ \/ |/ /
 /    / , _/ // / // // /  / / / , _/ /_/ /    / 
/_/|_/_/|_/____/____/___/ /_/ /_/|_|\____/_/|_/  
                                                 
```                                              

<i>"Let's do some science."</i>



Wait, why's this a Jupyter notebook? Great question - I'm not sure.

Todo: 
- Chamber assembly price 
- distance travelled per period
- ICP skin depth
- [] Optimize number of turns to current
- [] enthalpy of vaporization


In [1]:
%matplotlib nbagg

import matplotlib.pyplot as plt

from mendeleev import element
from thermo.chemical import Chemical
import sympy.physics.units as units
import scipy.constants as constants
amu = constants.physical_constants["atomic mass unit-kilogram relationship"][0]
import math
import sys
import numpy as np

sys.path.insert(0, '../../files/ionprinter/simulation/utilities/')
import util


# Inputs

In [2]:
#GLOBALS

print_species = element('Aluminum')

print_speed_g_h = 10.0 #g/hour

#convert to kg/s, kick CGS to the curb.
print_speed = print_speed_g_h/1000.0
print_speed /= 3600.0

number_of_bowties = 250000 #bowties/printer - an excellent unit

#hold or cut bowties

build_platform_distance = 0.03 #m

#BOWTIE

bowtie_nozzle_radius = 0.00033#m

bowtie_hot_area = 1e-6 #m^2
graphite_emissivity = 0.95

bowtie_temperature = 1400.0 #Kelvin


#CHAMBER

ionization_chamber_radius = 0.003 #m
ionization_chamber_length = 0.01 #m

acceleration_gap = 0.00025 #m
acceleration_voltage = 100 #V
ionization_efficiency = 0.3 

ICP_frequency = 1000000 #hz
ICP_peak_B = 0.05 #T

# CATHODE

cathode_power_consumption = 17.5 #"efficiency", watts per amp!
cathode_wire_diameter = 0.0001 #m
cathode_emission = 4.0

#IONIZATION E-BEAM

e_beam_energy = 100.0 #eV


# import pandas as pd
# data = [[1, 2], [3, 4]]
# pd.DataFrame(data, columns=["Foo", "Bar"])

R_constant = constants.Boltzmann*constants.Avogadro


# Vacuum level

PV=NRT to get chamber number density, then MFP.

In [3]:
# vacuum_gas_atomic_radius = 65e-9 #nitrogen
# vacuum_gas_cross_section = math.pi*(chamber_gas_atomic_radius**2.0)
# vacuum_gas_pressure = 1e-4 #Pa

# chamber_number_density (vacuum_gas_pressure/(273.15))=N

# ionization_electron_mean_free_path = 1.0/(nozzle_number_density*)

# ionization_electrons_per_neutral = bowtie_nozzle_radius/ionization_electron_mean_free_path

# Neutral stuff.

In [4]:
#Total n of atoms that must be emitted to achieve desired mass deposition per hour
atoms_per_second = print_speed/(print_species.mass*amu)

moles_per_second = atoms_per_second/constants.Avogadro

#Volume deposited per hour.
hourly_print_volume = ((print_speed*3600.0)/(print_species.density*1000.0))/1.0e-6

total_beam_current = atoms_per_second*constants.elementary_charge
per_chamber_beam_current = total_beam_current/number_of_bowties

print("Atoms per second: \nper chamber %0.5g atoms/s \ntotal %0.5g atoms/s\n" % ((atoms_per_second/number_of_bowties),
                                           atoms_per_second))
print("Total moles per second: %0.5g M/s" % moles_per_second)
print("Deposited volume: %0.5g cm^3/h" % hourly_print_volume)
print("Total beam current: %0.5g Amps" % total_beam_current)
print("Per-chamber current: %0.5g Amps" % per_chamber_beam_current)

#I find it infinitely amusing that "40 amps of aluminum" is "40 grams per second of aluminum" within 0.6%

Atoms per second: 
per chamber 2.4799e+14 atoms/s 
total 6.1999e+19 atoms/s

Total moles per second: 0.00010295 M/s
Deposited volume: 3.7052 cm^3/h
Total beam current: 9.9333 Amps
Per-chamber current: 3.9733e-05 Amps


## RMS neutral velocity

Determine RMS neutral gas velocity at a certain temperature:

$$\sqrt{\frac{3RT}{m}}$$

In [5]:
bowtie_rms_neutral_velocity = math.sqrt((3*constants.Boltzmann*bowtie_temperature)/(print_species.mass*amu))
bowtie_rms_energy = 0.5*print_species.mass*amu*(bowtie_rms_neutral_velocity**2.0)
print("RMS particle velocity: %0.5g m/s" % bowtie_rms_neutral_velocity)
print("RMS particle energy: %0.5g eV" % (bowtie_rms_energy/constants.electron_volt))

RMS particle velocity: 1137.6 m/s
RMS particle energy: 0.18096 eV


## Bowtie nozzle gas pressure

Determine approximate neutral gas pressure in a virtual volume "1 thermal velocity" long via a simple 

$$\text{PV}=\text{nRT}$$

(I'm not totally sure that you can do this.)

Also approximately determine mean free path with the neutral Van Der Waals radius.

In [6]:

#volume nozzle area times 1 thermal neutral injection velocity long
nozzle_virtual_volume = (math.pi*(bowtie_nozzle_radius**2))*bowtie_rms_neutral_velocity

bowtie_nozzle_pressure = ((moles_per_second/number_of_bowties)*R_constant*bowtie_temperature)/(nozzle_virtual_volume)

mean_free_path = 1.0/(((atoms_per_second/number_of_bowties)/nozzle_virtual_volume)*math.pi*((print_species.vdw_radius*1e-12)**2))

nozzle_number_density = (atoms_per_second/nozzle_virtual_volume)

print("Bowtie nozzle pressure: %0.5g Pa" % bowtie_nozzle_pressure)
print("Approximate nozzle mean free path: %0.5g m" % mean_free_path)

#number density of atoms within that virtual volume - useful for DSMC stuff
print("nrho: %0.5g" % (nozzle_number_density))

print("Knudsen number: local %0.5g, global %0.5g" % ((mean_free_path/bowtie_nozzle_radius),
                                                     (mean_free_path/build_platform_distance)))

print("Worst-case space charge density: %0.5g" % ((atoms_per_second/number_of_bowties)/nozzle_virtual_volume))

Bowtie nozzle pressure: 0.012316 Pa
Approximate nozzle mean free path: 14.756 m
nrho: 1.5929e+23
Knudsen number: local 44714, global 491.86
Worst-case space charge density: 6.3717e+17


## Actual vapor pressure

Now a rough approximation of the actual print species vapor pressure one can expect to find at that temperature:

(likely off by at least 50% - https://www.iap.tuwien.ac.at/www/surface/vapor_pressure is a more accurate source)

In [7]:
vapor_pressure = Chemical(print_species.name).VaporPressure.calculate(bowtie_temperature,"BOILING_CRITICAL") # takes temp in K, returns pressure in P
print("Approx. species vapor pressure: %0.5g Pa" % vapor_pressure)

Approx. species vapor pressure: 0.47493 Pa


## Bowtie radiation power loss

Bowtie IR/Vis radiation power by the Stefan-Boltzmann law:

In [8]:
per_bowtie_power = (bowtie_hot_area * graphite_emissivity * constants.Stefan_Boltzmann * (bowtie_temperature**4.0))
total_bowtie_power = per_bowtie_power*number_of_bowties

print("Bowtie power: per %0.5g W, total %0.5g W" % (per_bowtie_power,total_bowtie_power))


Bowtie power: per 0.20694 W, total 51735 W


# Acceleration

In [9]:
acceleration_velocity = math.sqrt((2.0*acceleration_voltage*constants.electron_volt)/(print_species.mass*amu))

print("Accel. vel: %0.5g m/s" % acceleration_velocity)
print("Accel. power per chamber: %0.5g W, total %0.5g W " % (acceleration_voltage*per_chamber_beam_current
                                                        ,acceleration_voltage*total_beam_current))



Accel. vel: 26743 m/s
Accel. power per chamber: 0.0039733 W, total 993.33 W 


# Ionization

### Power
Simple first-ionization power consumption estimate:

In [10]:
per_chamber_ionization_power = ((atoms_per_second/number_of_bowties)*print_species.ionenergies[1]*constants.electron_volt)

total_ionization_power = per_chamber_ionization_power*number_of_bowties

ionization_chamber_area = math.pi*(ionization_chamber_radius**2.0)

print("Ionization power: per ~%0.5g W, total %0.5g W" % (per_chamber_ionization_power,total_ionization_power))

Ionization power: per ~0.00023783 W, total 59.458 W


### E-beam impact

An equation that accurately describes ionization cross section in relation to electron energy does not appear to have been developed.

Here's a paper from antiquity that describes the situation quite well.

Lotz, Wolfgang. "Electron-impact ionization cross-sections for atoms up to Z= 108." Zeitschrift für Physik A Hadrons and nuclei 232.2 (1970): 101-107.

https://link.springer.com/content/pdf/10.1007/BF01393132.pdf

These values can be assumed to be half an order of magnitude off.

If we assume a transverse beam of electrons, 

$$ \lambda=\frac{1}{\rho\sigma}$$

where $\sigma$ is the collision cross section, $\rho$ is the local number density (neutral atoms/m^3), and $\lambda$ is the electron MFP.

We can multiply this MFP by the beam diameter (simplifying to a square beam, of course) to get the odds that a certain electron will collide with a particle in the beam.



This is vastly oversimplified, of course - we're ignoring the townsend secondaries entirely, second- and third- ionizations are likely, and there will probably be some weird space charge compensation effects from the nearby ionized particles - but we've got E-beam current to spare.

A measurement of the ionization cross-section
of helium ions by electron impact

THE IONIZATION OF HELIUM, NEON,
AND ARGON BY ELECTRON IMPACT 

Todo: calculate values per-element based on electron configuration

An even better reference: @kim2005electron, a nist database

### First ionization

In [11]:
impact_cross_section = 3.0e-20 #m^2
#value for aluminum approx 3.0e-20

ionization_electron_mean_free_path = 1.0/(nozzle_number_density*impact_cross_section)

ionization_electrons_per_neutral = ionization_electron_mean_free_path/bowtie_nozzle_radius

print("Ionization mean-free-path ~%0.5gm" % (ionization_electron_mean_free_path))
print("Required e-beam to heavy-beam factor: ~%0.5gx " % (ionization_electrons_per_neutral))

Ionization mean-free-path ~0.00020926m
Required e-beam to heavy-beam factor: ~0.63412x 


Clearly the e-beam to heavy-beam ratio can't be < 1 unless secondaries are included, so this is an underestimate.

### Second ionization 

"Electron-Impact Ionization Cross-Sections and Ionization Rate Coefficients for Atoms and Ions" http://adsbit.harvard.edu//full/1967ApJS...14..207L/0000211.000.html

The second ionization cross sections seem to 

In [12]:
# second_impact_cross_section = 3.0e-20 #m^2

# ionization_electron_mean_free_path = 1.0/(nozzle_number_density*impact_cross_section)

# ionization_electrons_per_neutral = bowtie_nozzle_radius/ionization_electron_mean_free_path

# print("Ionization mean-free-path ~%0.5gm" % (ionization_electron_mean_free_path))
# print("Required e-beam to heavy-beam factor: ~%0.5gx " % (ionization_electrons_per_neutral))

(old, ignore) ICP acceleration field, (line integral of Maxwell's second equation, eq. 4.5-3 of "Fundamentals"):

In [13]:
# (-(1.0j*2.0*math.pi*ICP_ionization_frequency)/2.0)*

In [14]:
specific_emission = 4 #amps/cm^2
specific_emission /= 0.0001

required_area = per_chamber_beam_current/specific_emission
required_area = required_area * 1000000.0 #mm^2
required_area


0.0009933266541915016

# Ion beam deflection / scattering by e-beam collision:

Basic momentum transfer upper bound. Ignoring the energy of ejected secondary electron and all other important factors

In [15]:
heavy_beam_collision_delta_v = ((math.sqrt((2.0*e_beam_energy*constants.electron_volt)/constants.electron_mass))*constants.electron_mass)/(print_species.mass*amu)
heavy_beam_deflection_angle = math.degrees(math.atan(heavy_beam_collision_delta_v/bowtie_rms_neutral_velocity))

print("Heavy beam will have up to <%0.5g m/s added" % (heavy_beam_collision_delta_v))
print("Heavy beam will be deflected by <%0.5g degrees" % (heavy_beam_deflection_angle))

Heavy beam will have up to <120.59 m/s added
Heavy beam will be deflected by <6.0505 degrees


 ## Assembly 
 
 Charge assembly energy, circular approximation (eq. 8.7 FLP Vol II):
 
 TODO: add cylindrical eq. - this is horribly inaccurate, but at least it's an upper bound.
 
 FIXME:
 
 Total charge:
 $$ Q_T = \text{pL}\pi\text{R}^2$$
 
 
 
 

In [16]:
ionization_chamber_volume = (math.pi*(ionization_chamber_radius**2))*bowtie_rms_neutral_velocity


per_chamber_assembled_energy = (3.0/5.0)*(((atoms_per_second/number_of_bowties)* \
                                 (ionization_chamber_radius/bowtie_rms_neutral_velocity)*constants.e)**2.0) \
                                /(4.0*math.pi*constants.epsilon_0*ionization_chamber_radius)*(bowtie_rms_neutral_velocity/ionization_chamber_radius)

print("Charge assembly power: per ~%0.5g W, total %0.5g W" % (per_chamber_assembled_energy, per_chamber_assembled_energy*number_of_bowties))

Charge assembly power: per ~0.0074832 W, total 1870.8 W


Energy to form one sphere of charge:

$$ U=(3/5)\left(\frac{\left(\frac{\text{I C}}{\frac{v}{2R}}\right)^2}{4\pi\epsilon_0 R}\right) $$

simplifies to

$$ (3/5)(Q^2 R)/(π v^2 ϵ) $$

Again from http://www.feynmanlectures.caltech.edu/II_08.html


```
I = beam current, amps
C = Coulomb's constant
v = particle velocity, post-acceleration
R = final beam radius
```

In [17]:
#Take two:
#that can't be right, it's independent of beam radius
#which is absurd
minimum_beam_radius = 0.001
per_chamber_assembled_energy = (3.0/5.0)*(((constants.e*(atoms_per_second/number_of_bowties))**2.0)*minimum_beam_radius)/(math.pi*(bowtie_rms_neutral_velocity**2.0)*constants.epsilon_0)*(bowtie_rms_neutral_velocity/minimum_beam_radius)
assembled_beam_energy_per_particle = ((per_chamber_assembled_energy/(atoms_per_second/number_of_bowties))/constants.electron_volt)
per_chamber_assembled_energy
assembled_beam_energy_per_particle

753.3493486763224

Take 3:
Dividing the minimum beam volume by each sphere volume, then multiplying by the beam charge per second:

$$(((electron charge * (3v/4r)*(total beam charge))^2)/R) * (1/(4pi \epsilon_0)) * (3/5) $$



In [18]:
per_chamber_assembled_energy = (3.0/5.0)*(((((constants.e*(atoms_per_second/number_of_bowties))*((3.0*bowtie_rms_neutral_velocity)/(4.0*minimum_beam_radius)))**2.0)/minimum_beam_radius)/(4.0*math.pi*constants.epsilon_0))*(bowtie_rms_neutral_velocity/minimum_beam_radius)
per_chamber_assembled_energy


7.050895478480679e+21

Moayedi, Seyed Kamran, Mansoureh Shafabakhsh, and Farkhondeh Fathi. "Analytical calculation of stored electrostatic energy per unit length for an infinite charged line and an infinitely long cylinder in the framework of Born-Infeld electrostatics." Advances in High Energy Physics 2015 (2015).

Another cylindrical approximation, from the last equation on

https://nptel.ac.in/courses/115101005/downloads/lectures-doc/Lecture-10.pdf



In [19]:
linear_charge_density = ((atoms_per_second/number_of_bowties)*constants.e)/acceleration_velocity
per_chamber_assembled_energy = ((linear_charge_density**2.0)/(16.0*math.pi*constants.epsilon_0))*acceleration_velocity
per_chamber_assembled_energy*number_of_bowties
assembled_beam_energy_per_particle = ((per_chamber_assembled_energy/(atoms_per_second/number_of_bowties))/constants.electron_volt)
assembled_beam_energy_per_particle

3.338268021664013

Electron-heavy space charge compensation test:

Complete compensation requires that the two beams have equal linear charge densities.

$$ \frac{I_e e^-}{\sqrt{2E_e/m_e}} = \frac{I_b q_b}{\sqrt{2E_b/m_b}} $$

The heavy beam current and energy are quite fixed; the only free variables are the E-beam current and energy.



# Child-Langmuir Ion Current Limit

$$ \text{I} = \frac{4}{9}\epsilon_0 \left(\frac{-2q}{m} \right) ^{0.5} \frac{\text{V}^{1.5}}{ \text{D}^2} \text{A} $$

```
I = beam current, amps
V = acceleration voltage, volts
q = ion charge, C
m = ion mass, kg
D = gap between acceleration plates, m
A = area

```

Suggested reading:

Fundamentals of Electric Propulsion, "Basic plasma physics". ../references/Fundamentals.pdf

The Child-Langmuir Law, Stanford EDU, Lucas et al [1]

[2]

[1]: https://web.stanford.edu/~ajlucas/The%20Child-Langmuir%20Law.pdf

[2]: http://www.physics.csbsju.edu/370/thermionic.pdf

"A simple physical derivation of Child–Langmuir space-charge-limited
emission using vacuum capacitance"


In [20]:
def child_langmuir_current(voltage, charge, mass, gap, area):
    return (4.0/9.0)*constants.epsilon_0*(((2.0*charge/(mass))**0.5)*((voltage**1.5)/(gap**2.0))) * area
                                   
per_chamber_CL_ion_current = child_langmuir_current(acceleration_voltage,constants.e,(print_species.mass*amu),acceleration_gap,ionization_chamber_area)
                                   
print("Child-langmuir ion current: %0.5g A/bowtie" % per_chamber_CL_ion_current)
print("Total CL current: %0.5g A" % (per_chamber_CL_ion_current*number_of_bowties))


Child-langmuir ion current: 0.0047609 A/bowtie
Total CL current: 1190.2 A


2d ion 

Lau, Y. Y. "Simple theory for the two-dimensional Child-Langmuir law." Physical review letters 87.27 (2001): 278301.

# Space charge


In [21]:
# Vkick

https://cds.cern.ch/record/1005034/files/p27.pdf 



https://arxiv.org/pdf/1401.3951.pdf

Three-dimensional Child–Langmuir law for uniform hot electron emission

In [22]:
beam_exit_velocity = bowtie_rms_neutral_velocity+acceleration_velocity

focus_field = util.scharge_efield(per_chamber_beam_current,beam_exit_velocity,ionization_chamber_radius)
print("Required focus field: %0.5g V/m" % focus_field)

#sine focus voltage equation would be nice
print("Required focus voltage: ~%0.5g V" % (focus_field*(acceleration_gap)))


Required focus field: 8542.8 V/m
Required focus voltage: ~2.1357 V


In [23]:
#todo: 2d scharge from 

# Warp acceleration gap sim

This required a modification to warp - line 28 of warp.py needs to have 
```
try:
    __IPYTHON__
    pass
except NameError:
    warpoptions.parse_args()
```

In [24]:
# from warp import *

# w3d.solvergeom = w3d.RZgeom

# w3d.bound0 = dirichlet
# w3d.boundnz = neumann
# w3d.boundxy = neumann
# # ---   for particles
# top.pbound0 = absorb
# top.pboundnz = absorb
# top.prwall = ionization_chamber_radius

# # --- Set field grid size
# w3d.xmmin = -ionization_chamber_radius
# w3d.xmmax = +ionization_chamber_radius
# w3d.ymmin = -ionization_chamber_radius
# w3d.ymmax = +ionization_chamber_radius
# w3d.zmmin = 0.
# w3d.zmmax = ionization_chamber_length

# # --- Field grid dimensions - note that nx and ny must be even.
# w3d.nx = w3d.ny = 32
# w3d.nz = 32

# f3d.mgtol = 1.e-1 # Multigrid solver convergence tolerance, in volts


# plate = ZRoundedCylinderOut(radius=0.0001, length=0.0001, radius2=0.00001, voltage=100.0, zcent=0.01)

# installconductor(plate,dfill=largepos)

# solver = MultiGrid2D()
# registersolver(solver)

# package("w3d")
# generate()

# step()

# getappliedfields(x=np.arange(-ionization_chamber_radius,ionization_chamber_radius,0.001),
#                  y=np.arange(-ionization_chamber_radius,ionization_chamber_radius,0.001),
#                  z=0.0)


# Recombination

## Child-Langmuir recombination current


In [25]:
per_chamber_CL_electron_current = child_langmuir_current(5,constants.e,constants.electron_mass,acceleration_gap,ionization_chamber_area)

print("Child-langmuir electron current: %0.5g A/bowtie" % per_chamber_CL_electron_current )
print("Total CL e current: %0.5g A" % (per_chamber_CL_electron_current*number_of_bowties))


Child-langmuir electron current: 0.011805 A/bowtie
Total CL e current: 2951.2 A


https://casper.ssl.berkeley.edu/astrobaki/index.php/Recombination_Coefficients

$${\textrm{# of recombinations}\over \textrm{volume} \cdot \textrm{time} }= n_+n_e{\sigma v}$$

Where `v` is relative velocity of the two particles, and n+ and ne are the number of electrons and 

The capture cross section for thermal electrons (cm^2, as given by @ biondi1949measurement, page 5) is as follows:

$$\sigma = 8.64\left(\frac{Z_{\text{effective}}^{8/5}e^4}{\text{m}_e^2 c^{(6/5)}v^{(14/5)}}\right)$$

> where a- is the capture cross section; Ze is the effective charge of the ion; e and m are the electron charge and mass, respectively, and c is the velocity of light.

> Using the quantum mechanical model of the helium ion and the
Thomas-Fermi model for the neon and argon ions, we may calculate the effective ion
charge Ze at the distance of closest approach of the electron to the ion. For thermal
electrons, the value of Ze for helium is approximately 2; for neon, it lies between 8
and 9; and for argon, between 11 and 13. Since most of the energy is radiated while the
electron is close to the ion, the use of these values of Ze in Eq. (1) introduces only a
small error in the calculation. 

Even the best models of recombination seem to be quite inaccurate: theory often differs with reality by between 10 to 1000x. This will be a lower bound.

I'm not sure whether the velocities are supposed to be m/s or cm/s.

In [26]:
Z_e = 2.0 #depends on print species
# differential_energy = 0.00001
differential_velocity = 1
# differential_velocity = math.sqrt((2.0*differential_energy*constants.electron_volt)/(print_species.mass*amu))
# v matching of 1 part in 1000 seems achievable - just a guess though
# matching will be limited by ion and electron thermal velocity

#cm^2
cross_section = 8.64*(((Z_e**(8.0/5.0))*(constants.e**4.0))/((constants.electron_mass**2.0)*(constants.c**(6.0/5.0))*(differential_velocity**(14.0/5.0))))
# cross_section *= 0.0001 #convert to m^2
cross_section



recombination_electron_mean_free_path = 1.0/(nozzle_number_density*cross_section)

recombination_region_length = 0.001 #mm

recombination_electrons_per_neutral = recombination_electron_mean_free_path/recombination_region_length

print("Ionization mean-free-path ~%0.5gm" % (recombination_electron_mean_free_path))
print("Required e-beam to heavy-beam factor: ~%0.5gx" % (recombination_electrons_per_neutral))

Ionization mean-free-path ~4.4871m
Required e-beam to heavy-beam factor: ~4487.1x


In [27]:
Z_e = 2.0 #depends on print species
# differential_energy = 0.00001
differential_velocity = 0.1
# differential_velocity = math.sqrt((2.0*differential_energy*constants.electron_volt)/(print_species.mass*amu))
# v matching of 1 part in 1000 seems achievable - just a guess though
# matching will be limited by ion and electron thermal velocity

#cm^2
cross_section = 8.64*(((Z_e**(8.0/5.0))*(constants.e**4.0))/((constants.electron_mass**2.0)*(constants.c**(6.0/5.0))*(differential_velocity**(14.0/5.0))))
# cross_section *= 0.0001 #convert to m^2
cross_section



recombination_electron_mean_free_path = 1.0/(nozzle_number_density*cross_section)

recombination_electrons_per_neutral = recombination_electron_mean_free_path/bowtie_nozzle_radius

print("Ionization mean-free-path ~%0.5gm" % (recombination_electron_mean_free_path))
print("Required e-beam to heavy-beam factor: ~%0.5gx" % (recombination_electrons_per_neutral))

Ionization mean-free-path ~0.0071115m
Required e-beam to heavy-beam factor: ~21.55x


## Hot cathode

# Part heating

The beam heat flux will cause a temperature gradient from the chilled build platform to the extreme end of the printed part. Assuming a constant cross section, the peak temperature will be

$$ \frac{\text{(P/A)}}{k}L = \Delta \text{T} \\ \Delta \text{T} + \text{T}_i = T $$

```
P = Power applied at surface, W
A = Area of the part, m^2
k = Thermal conductivity of material, W/m-K
L = Length, M
T = Change in temperature across material, K
Ti = Build platform temperature
```

Ionization power is likely emitted in the form of photons at recombination, and so should be neglected.


http://web.mit.edu/16.unified/www/SPRING/propulsion/notes/node116.html

TODO: 
- Add radiation here. A bit tricky, since the temperature's not constant.
- Add some kind of duty cycle approximation?
- cool graph

In [28]:
print_area = 0.01*0.1 #m^2
printed_object_height = 0.03 #m
build_platform_temperature = 200.0 #Kelvin - likely liquid-cooled 

printed_emissivity = 0.05
beam_power = (acceleration_voltage*total_beam_current)

part_peak_temperature = (((beam_power/print_area)/print_species.thermal_conductivity)*printed_object_height)+build_platform_temperature
print("Part peak temperature: %0.5g K" % (part_peak_temperature))


# per_bowtie_power = (bowtie_hot_area * printed_emissivity * constants.Stefan_Boltzmann * (bowtie_temperature**4.0))


Part peak temperature: 325.74 K


# A final tally

In [29]:
total_power_consumption = total_bowtie_power + (acceleration_voltage*total_beam_current) + total_ionization_power
print("Total power consumption: %0.5g W" % (total_power_consumption))

single_chamber_assembly_area = math.pi * (0.005**2.0)

print("Print head area: %0.5g m^2 (a square %0.5g m to the side)" % ((single_chamber_assembly_area*number_of_bowties),((single_chamber_assembly_area*number_of_bowties)**0.5)))

print("Allowable power consumption per bowtie")

Total power consumption: 52788 W
Print head area: 19.635 m^2 (a square 4.4311 m to the side)
Allowable power consumption per bowtie


In [30]:
#economics!

cost_per_watt = 0.2/1000.0 #20 cents per kilowatt-hour

consumable_cost = 200.0/1000.0 #dollars per 1000 hours

material_cost = 20.0/1000.0 #dollars per gram

joules_per_gram = total_power_consumption/print_speed_g_h

print("Print cost: %0.5g J/g, $%0.5g/100g" % ((joules_per_gram),(cost_per_watt*joules_per_gram*100.0+material_cost+consumable_cost)))


Print cost: 5278.8 J/g, $105.8/100g
